In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+'1'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')


In [3]:
trs = soup.select('form>table>tbody>tr')
len(trs)

100

In [4]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+'1'
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [6]:
len(driver.page_source), len(res.text)

(296417, 292070)

- Step 3

In [7]:
tr = trs[0]

In [8]:
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [9]:
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [10]:
channel = tr.select_one('.subject>h1>a').get_text().strip()
channel

'BLACKPINK'

In [11]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'8730만'

In [12]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view, video

('305억3350만', '507개')

In [13]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [14]:
convert('335억8260만')

33582600000

- Step 4

In [15]:
lines= []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject>h1>a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    lines.append({
        '순위':rank, '카테고리':category, '채널명':channel,
        '구독자수':subscriber, '조회수':view, '영상수':video
    })

In [16]:
df1 = pd.DataFrame(lines)
df1.tail()

,순위,카테고리,채널명,구독자수,조회수,영상수
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450
99,100,미분류,매일맛나 delicious day,4740000,482360000,325


In [17]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+'2'
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [22]:
lines= []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+ str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')
    
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject>h1>a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({
            '순위':rank, '카테고리':category, '채널명':channel,
            '구독자수':subscriber, '조회수':view, '영상수':video
        })
driver.close()

In [23]:
df = pd.DataFrame(lines)
df.shape


(1000, 6)

In [24]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,영상수
995,991,회사/오피셜,LG Uplus,730000,1379980000,2473
996,998,BJ/인물/연예인,케이,730000,572670000,3304
997,1001,미분류,백크 Baekk,720000,457320000,1106
998,999,음악/댄스/가수,Crush,720000,395460000,131
999,997,BJ/인물/연예인,bobaepapa - 보배아빠,720000,348440000,393


In [25]:
df.to_csv('data/유튜브랭킹.csv',index=False)